# Touching Properties

This notebook provides a quick demonstration of what can happen if you don't "touch" properties prior to solving the model and further explanation of when touching property variables is necessary. "Touching" a property is simply a line of code that attempts to access the desired property. For example, "touching" osmotic pressure is simply, e.g.

```python
    m.fs.unit.properties[0].pressure_osm_phase
```

This demonstration is for `pressure_osm_phase`, but other properties in the seawater property model that would require touching are:

```python
    mass_frac_phase_comp
    dens_mass_phase_comp
    dens_mass_solvent
    flow_vol_phase
    flow_vol
    conc_mass_phase_comp
    visc_d_phase
    osm_coeff
    enth_mass_phase
    enth_flow
    pressure_sat
    cp_mass_phase
    therm_cond_phase
    dh_vap_mass
    diffus_phase_comp
    boiling_point_elevation_phase
    flow_mol_phase_comp
    mole_frac_phase_comp
    molality_phase_comp
```

To touch *indexed* variables (e.g., `conc_mass_phase_comp`), touching with any index (or no index) will have the same effect. For example, the following two lines of code are equivalent for the purposes of touching `conc_mass_phase_comp`

```python

    m.fs.unit.properties[0].conc_mass_phase_comp
    m.fs.unit.properties[0].conc_mass_phase_comp["Liq", "TDS"]
```

Importantly, touching property variables is not necessary for unit model state blocks that use the property. For example, the RO model requires the use of `pressure_osm_phase` on the *inlet* stream so you would not need to touch that property for that state block. *But*, if you were interested in knowing the osmotic pressure of the permeate stream, you would need to touch the property on the permeate state block prior to solving.

In [ ]:
from pyomo.environ import ConcreteModel, value

from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock

from watertap.core.solvers import get_solver

solver = get_solver()

## Two Build Functions

Below are two build functions. One touches both `pressure_osm_phase` and `conc_mass_phase_comp` prior to solving. The other only touches `conc_mass_phase_comp`. 

Running this cell will create, initialize, and solve both models, and then print out the concentration and osmotic pressure for both.

In [ ]:
def touch_osmotic():
    """
    Create feed model using seawater property package
    Touch osmotic pressure and mass concentration prior to solving
    """
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    m.fs.properties = SeawaterParameterBlock()
    m.fs.feed = Feed(property_package=m.fs.properties)
    m.fs.feed.properties[0].pressure_osm_phase
    m.fs.feed.properties[0].conc_mass_phase_comp
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
    m.fs.feed.properties[0].temperature.fix(273 + 25)
    m.fs.feed.properties[0].pressure.fix(101325)

    print(f"Model 1 degrees of freedom: {degrees_of_freedom(m)}\n")

    # Initialize and solve for the initial conditions
    m.fs.feed.initialize()
    results = solver.solve(m)
    print(f"Model 1 termination {results.solver.termination_condition}")

    return m


def dont_touch_osmotic():
    """
    Create feed model using seawater property package
    Only touch mass concentration prior to solving
    """
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    m.fs.properties = SeawaterParameterBlock()
    m.fs.feed = Feed(property_package=m.fs.properties)
    m.fs.feed.properties[0].conc_mass_phase_comp
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
    m.fs.feed.properties[0].temperature.fix(273 + 25)
    m.fs.feed.properties[0].pressure.fix(101325)

    print(f"Model 2 degrees of freedom: {degrees_of_freedom(m)}\n")

    # Initialize and solve for the initial conditions
    m.fs.feed.initialize()
    results = solver.solve(m)
    print(f"Model 2 termination {results.solver.termination_condition}")

    return m


m1 = touch_osmotic()

print(f"\nResults with touching properties:")
print(
    f"\tConcentration: {value(m1.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.2f} g/L"
)
print(
    f"\tOsmotic pressure: {value(m1.fs.feed.properties[0].pressure_osm_phase['Liq'])/1e5:.2f} bar\n"
)

m2 = dont_touch_osmotic()

print(f"\nResults without touching properties:")
print(
    f"\tConcentration: {value(m2.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.2f} g/L"
)
print(
    f"\tOsmotic pressure: {value(m2.fs.feed.properties[0].pressure_osm_phase['Liq'])/1e5:.2f} bar\n"
)

### The second model results are different

Both models give the same concentration of 35.83 g/L, but the second model gives an osmotic pressure of 10 bar, which we know is incorrect.

Because we didn't touch the property prior to `results2 = solver.solve(m2)`, the value being printed is the _initial value_ of the variable `m.fs.feed.properties[0].pressure_osm_phase['Liq']`.

_Notably_, the very act of trying to access the variable in the print statement above "touches" the variable, and a re-solve of the model will produce the correct result.

In [ ]:
results2 = solver.solve(m2)

print(f"Re-solve termination {results2.solver.termination_condition}")
print(
    f"\n\tConcentration: {value(m2.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.2f} g/L"
)
print(
    f"\tOsmotic pressure: {value(m2.fs.feed.properties[0].pressure_osm_phase['Liq'])/1e5:.2f} bar\n"
)